# COVID-19 Data (API).
### COVID-19 Country Data.
[site](https://covid19api.com/) | [API info](https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest)


In [0]:
# necessary imports for notebook.
import pandas as pd
import requests
import json
import plotly.graph_objects as go

In [4]:
''' COVID-19 : Country/State/Province Daily Individual Case'''
# COVID-19 Data.
'''https://covid19api.com/'''
'''https://documenter.getpostman.com/view/10808728/SzS8rjbc?version=latest'''
response = requests.get("https://api.covid19api.com/dayone/country/us/status/confirmed")
# show the status code.
print(response.status_code)
covid_ci = response.json()
covid_countryindividual = pd.json_normalize(covid_ci)
# show the data frame shape.
print(covid_countryindividual.shape)
# show the data frame with headers.
covid_countryindividual.head()

200
(123207, 10)


,Country,CountryCode,Province,City,CityCode,Lat,Lon,Cases,Status,Date
0,United States of America,US,,,,37.09,-95.71,1,confirmed,2020-01-22T00:00:00Z
1,United States of America,US,,,,37.09,-95.71,1,confirmed,2020-01-23T00:00:00Z
2,United States of America,US,Washington,King,53033,47.49,-121.83,1,confirmed,2020-01-23T00:00:00Z
3,United States of America,US,Washington,King,53033,47.49,-121.83,1,confirmed,2020-01-24T00:00:00Z
4,United States of America,US,,,,37.09,-95.71,2,confirmed,2020-01-24T00:00:00Z


In [5]:
# new data frame.
df = covid_countryindividual.copy()
# show the data frame shape.
print(df.shape)
# show the data frame with headers.
df.head()

(123207, 10)


,Country,CountryCode,Province,City,CityCode,Lat,Lon,Cases,Status,Date
0,United States of America,US,,,,37.09,-95.71,1,confirmed,2020-01-22T00:00:00Z
1,United States of America,US,,,,37.09,-95.71,1,confirmed,2020-01-23T00:00:00Z
2,United States of America,US,Washington,King,53033,47.49,-121.83,1,confirmed,2020-01-23T00:00:00Z
3,United States of America,US,Washington,King,53033,47.49,-121.83,1,confirmed,2020-01-24T00:00:00Z
4,United States of America,US,,,,37.09,-95.71,2,confirmed,2020-01-24T00:00:00Z


In [0]:
# clean up the data.

# drop any latitude with 0.
df.drop(df[df['Lat'] == '0'].index, inplace = True)
# strip the minutes & seconds from the Date column.
df['Date'] = df['Date'].str.replace(r'T00:00:00Z', '').astype(object)
# change the data types of Lat & Lon to floats.
df = df.astype({'Lat': float})
df = df.astype({'Lon': float})

In [7]:
# final data frame.

# show the data frame shape.
print(df.shape)
# show the data frame with headers.
df.head()

(121883, 10)


,Country,CountryCode,Province,City,CityCode,Lat,Lon,Cases,Status,Date
0,United States of America,US,,,,37.09,-95.71,1,confirmed,2020-01-22
1,United States of America,US,,,,37.09,-95.71,1,confirmed,2020-01-23
2,United States of America,US,Washington,King,53033,47.49,-121.83,1,confirmed,2020-01-23
3,United States of America,US,Washington,King,53033,47.49,-121.83,1,confirmed,2020-01-24
4,United States of America,US,,,,37.09,-95.71,2,confirmed,2020-01-24


In [0]:
# set the mapbox style and access token.

''' style url '''
MAPBOX_STYLE = 'mapbox://styles/cvanchieri/ck922oal90bxw1ilzl8opoz7d'
''' access token '''
MAPBOX_TOKEN = 'pk.eyJ1IjoiY3ZhbmNoaWVyaSIsImEiOiJjazkwcTM3bGwwNGU4M2hucGYxNmkydGFzIn0.sDsBLsY2hw-6LPjKzL2ZqA'

In [0]:
# create the times and frames_data.
times = df.groupby(['Date'])['Date'].count().index.tolist()
frames_data = [df.loc[df['Date'] == t] for t in times]

In [0]:
# create the frames density plot.
frames = [go.Frame(data=[go.Densitymapbox(lat=f['Lat'], lon=f['Lon'], z=f['Cases'], radius=10)], name=str(f.iloc[0]['Date'])) for f in frames_data]

In [0]:
# create the mapbox attributes, play and pause button.
buttons=[
         dict(label="Play",method="animate",args=[None, {'fromcurrent':True, "transition": {"duration": 1, "easing": "quadratic-in-out"}}]),
         dict(label="Pause",method="animate",args=[[None], {"frame": {"duration": 0, "redraw": False},"mode": "immediate", "transition": {"duration": 0}}])
]
sliders_dict = {
    'active':0,
    'currentvalue': dict(font=dict(size=15), prefix='Date: ', visible=True),
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    'x': 0,
    'steps': []
}
for i,t in enumerate(times):
    slider_step = {"args": [
                        [t],
                        {"frame": {"duration": 300, "redraw": False},
                         #"mode": "immediate",
                         "transition": {"duration": 1, "ease": "quadratic-in-out"}}
                    ],
            "label": t,
            "method": "animate",
            "value": t
    }
    sliders_dict['steps'].append(slider_step)

In [0]:
# merge the mapbox and the data together and display.
fig = go.Figure(data = [go.Densitymapbox(lat=df['Lat'], lon=df['Lon'], z=df['Cases'], radius=5, colorscale='hot', zmax=5, zmin=0)],
               layout=go.Layout(updatemenus=[dict(type="buttons", buttons=buttons,showactive=True)] ), 
               frames=frames
)
fig.update_layout(mapbox_style=MAPBOX_STYLE, 
                  mapbox_accesstoken=MAPBOX_TOKEN,
                  mapbox_center_lon=-100,
                  mapbox_center_lat=38,
                  mapbox_zoom=3.5)
fig.update_layout(sliders=[sliders_dict],
                 title='USA Covid-19 Confirmed Cases Daily Count')
fig.update_layout(height=1000)
fig.show()

![Covid19MapBox](https://github.com/CVanchieri/CVanchieri.github.io/blob/master/img/Covid19ConfirmedMapBoxPost/Covid19ConfirmedMapBox.png?raw=true)

